# Load data from SQL, parse it appropriately

This script loads the data from the eICU database and parses the data for concepts required for the GOSSIS project. The script outputs the `eicu-gossis-data.csv` file for later use.

In [ ]:
from __future__ import print_function

import psycopg2
import os
import numpy as np
import pandas as pd

# cursors need to be rolled back if they fail
def execute_query_safely(sql, con):
    cur = con.cursor()
    
    # try to execute the query
    try:
        cur.execute(sql)
    except:
        # if an exception, rollback, rethrow the exception - finally closes the connection
        cur.execute('rollback;')
        raise
    finally:
        cur.close()
    
    return

import getpass

# prompt user for username/password for postgres database
host='localhost'
port=5647
print('Please enter your password for a PostgreSQL database with eICU.')
sqluser = getpass.getuser()
sqlpass = getpass.getpass(prompt='Username: {}\nPassword: '.format(sqluser))

## Connect to database

In [ ]:
print('Connecting to {} on port {} ...'.format(host,port))
con = psycopg2.connect(dbname='eicu', host=host, port=port, user=sqluser, password=sqlpass)


print('Connected to postgres {}.{}.{}!'.format(int(con.server_version/10000),
                                              (con.server_version - int(con.server_version/10000)*10000)/100,
                                              (con.server_version - int(con.server_version/100)*100)))

# default is to write to public and read from both public and eicu schema
query_schema = "set search_path to public,eicu_crd_phi;"

## Create initial cohort

In [ ]:
# read in file/create base cohort
f = 'sql/cohort.sql'
with open(f, 'r') as fp:
    query = ''.join(fp.readlines())

# Execute the query
print('Generating table using {} ...'.format(f),end=' ')
execute_query_safely(query_schema + query, con)
print('done.')

In [ ]:
# take a peek at the base cohort
query = query_schema + """select * from gossis_cohort"""
co = pd.read_sql_query(query,con)

# print out the exclusions
print('Cohort - initial size: {} ICU stays'.format(co.shape[0]))
idxRem = np.zeros(co.shape[0])
for c in co.columns:
    if c.startswith('exclusion_'):
        print('  {:5g} ({:2.2f}%) - {}'.format(np.sum(co[c]),np.mean(co[c])*100.0, c))
        idxRem[co[c].values==1] = 1
        
print('Final cohort size: {} ICU stays ({:2.2f}%).'.format(co.shape[0] - np.sum(idxRem), (1-np.mean(idxRem))*100.0))

## Create necessary tables

In [ ]:
# get a list of all SQL files in the subfolder
query_path = './sql/'
queries = [f for f in os.listdir(query_path) 
             # only keep the filename if it is actually a file (and not a directory)
            if os.path.isfile(os.path.join(query_path,f))
             # and only keep the filename if it is an SQL file
            & f.endswith('.sql')
            # and we do *not* want the cohort - it's generated above
            & (f != 'cohort.sql')
            # and we do *not* want the data - we generate that last
            & (f != 'data.sql')]

# sort the queries by name
# this helps ensure lab.sql is run before lab-d1.sql, etc
queries = sorted(queries)[::-1]

# execute each SQL file to generate the materialized views
for f in queries:
    print('Executing {} ...'.format(f), end=' ')
    
    with open(os.path.join(query_path,f)) as fp:
        query = ''.join(fp.readlines())
        
    execute_query_safely(query_schema + query, con)
        
    print('done.')

In [ ]:
# finally, generate the data
query_path = './sql/'
f='data.sql'
print('Executing {} ...'.format(f), end=' ')
with open(os.path.join(query_path,'data.sql')) as fp:
    query = ''.join(fp.readlines())
        
execute_query_safely(query_schema + query, con)
print('done.')

# 2 - Extract all covariates and outcome measures

We now query the aggregated data created by `data.sql`.

In [ ]:
# Load in the query from file
query = query_schema + """
--FINAL QUERY
select
  g.*
from gossis g
"""

# Load the result of the query into a dataframe
df = pd.read_sql_query(query, con)
print('Loaded data for {} patients and {} features.'.format(
        df.shape[0],
        df.shape[1]-2))

# hard code the data source as a field
df['data_source'] = 'eicu'

In [ ]:
# convert apache diagnosis into numeric ANZICS code using custom mapping table
dx_map = pd.read_csv('../etc/apache_diagnosis_map.csv',sep=',',dtype={'code': str})
dx_map.rename(columns = {'code': 'apache_3j_diagnosis'}, inplace=True)
df = df.merge(dx_map[['admitdiagnosis','apache_3j_diagnosis']],
              how='left',
              left_on='admitdiagnosis',
              right_on='admitdiagnosis')

In [ ]:
# add in apache 2 diagnosis using another mapping
ap_map = pd.read_csv('../etc/apache3-to-apache2.csv',sep=',',
                     dtype={'apache_3j_diagnosis': str, 'apache_2_diagnosis': str})

# create a column containing only the digits before '.' - this is the apache3 diagnosis
def get_ap3_code(x):
    if 'str' in str(type(x)):
        if '.' in x:
            return x.split('.')[0]
        else:
            return x
    else:
        return None
    
df['apache3dx'] = df['apache_3j_diagnosis'].map(get_ap3_code)

df = df.merge(ap_map,
              how='left', suffixes=('','_ap'),
              left_on='apache3dx',
              right_on='apache_3j_diagnosis')


# create the "apache_post_operative" column
df['apache_post_operative'] = (df['apache_3j_operative'] == 'Post-operative').astype(int)

In [ ]:
hdr = pd.read_csv('../hdr/header.csv',header=None,sep=',')[0].values

Map the data into a consistent header which is used for all databases. Warn if data is not found in the current dataset.

In [ ]:
df_new = pd.DataFrame()
for c in hdr:
    # did not find a mapping for the given variable
    if c not in df.columns:
        print('WARNING: {} not found in eICU data!'.format(c))
        df_new[c] = None
    else:
        # call the mapping
        df_new[c] = df[c]

# 3 - Output the data to a csv file

In [ ]:
df_new.to_csv('eicu-gossis-data.csv.gz',index=False,sep=',',compression='gzip')

In [ ]:
con.close()